In [7]:
import pandas
import math
import numpy as np

def get_res(DATASET, SKEW, PARTIES):
    experiment_directory = f"{DATASET}_non-IID_label_skew/{PARTIES}_parties/"

    distributions = []
    with open(f"{experiment_directory}individual/{SKEW}/{DATASET.lower()}_label_skew_{SKEW}_{PARTIES}clients_distribution.txt", "r") as reader:
        reader.readline()
        for i in range(PARTIES):
            distributions.append(reader.readline())

    weighted_mean = {'client_lr':0.0,
                    'client_momentum':0.0,
                    'batch_size':0}
    
    lr = np.zeros(PARTIES, dtype=float)
    momentum = np.zeros(PARTIES, dtype=float)
    batch_size = np.zeros(PARTIES, dtype=float)
    accuracy = np.zeros(PARTIES, dtype=float)

    # Get individual data and samples ratio
    for i in range(PARTIES):

        client_result = ""

        try:
            client_result = pandas.read_csv(f"{experiment_directory}individual/{SKEW}/{SKEW}_label_{PARTIES}clts_clt{i}.txt")
            #display(client_result.head(8).get(['val_accuracy','client_lr','client_momentum','round_epochs','batch_size']))
            #print(distributions[i])
            
            client_lr = client_result.head(1).get(['client_lr']).values[0][0]
            client_momentum = client_result.head(1).get(['client_momentum']).values[0][0]
            client_batch_size = client_result.head(1).get(['batch_size']).values[0][0]
            client_accuracy = client_result.head(1).get(['val_accuracy']).values[0][0]
            
            lr[i] = client_lr
            momentum[i] = client_momentum
            batch_size[i] = client_batch_size
            accuracy[i] = client_accuracy
                        
        except:
            print(f"File does not exist, client {i} crashed.")
    
    best_acc = np.max(accuracy)
    
    for i in range(PARTIES):
        #################  AGGREGATION METHOD BEGIN  #################
        weighted_mean['client_lr'] += lr[i] * (1-lr[i]) * (1-abs(best_acc-accuracy[i])) / PARTIES / 10
        weighted_mean['client_momentum'] += momentum[i]/PARTIES#min(1.0/PARTIES, momentum[i] * (1-momentum[i]) * (1-abs(best_acc-accuracy[i])) * 10)
        weighted_mean['batch_size'] += batch_size[i] * accuracy[i] / PARTIES
        #weighted_mean['client_lr'] += lr[i] / PARTIES
        #weighted_mean['client_momentum'] += momentum[i] / PARTIES
        #weighted_mean['batch_size'] += batch_size[i] / PARTIES
        #################  AGGREGATION METHOD END  #################
        
    # Get fedavg data
    try:
        with open(f"{experiment_directory}fedavg/{DATASET.lower()}_label_skew_{SKEW}_{PARTIES}clients.txt", "r") as reader:
            print("Federated average best parameters\n")

            for i in range(1):
                print(reader.readline())

    except:
        print("File does not exist")

    print(weighted_mean)

    # Get decentralized intervals search
    try:
        with open(f"{experiment_directory}individual/{SKEW}/{DATASET.lower()}_label_skew_{SKEW}_{PARTIES}clients_intervals.txt", "r") as reader:
            print("\nDecentralized intervals search\n")
            for i in range(PARTIES):
                print(reader.readline())
    except:
        print("File does not exist")
    
    # Get fedavg intervals search
    try:
        with open(f"{experiment_directory}intervals/intervals_{DATASET.lower()}_label_skew_{SKEW}_{PARTIES}clients.txt", "r") as reader:
            print("\nFederated average best intervals\n")

            for i in range(3):
                print(reader.readline())

    except:
        print("File does not exist")
    
DATASET= "SVHN_CROPPED" #["MNIST", "EMNIST", "SVHN_CROPPED", "CIFAR10"]
SKEW="0.1" #["0.1", "1.0", "5.0"] for label skew
PARTIES=10 #["10", "20"]
#get_res(DATASET, SKEW, PARTIES)
for p in [10, 20]:
    for s in ["0.1", "1.0", "5.0"]:
        get_res(DATASET, s, p)

Federated average best parameters

(DeviceArray(0.68922865, dtype=float32), {'act_fn': <jax.custom_derivatives.custom_jvp object at 0x7f479c587908>, 'client_lr': 0.07, 'server_lr': 0.03162277660168379, 'client_momentum': 0.3, 'server_momentum': 0.9, 'batch_size': 16, 'epochs_per_round': 2, 'rounds': 30, 'skew': 0.1})

{'client_lr': 0.011978981283068657, 'client_momentum': 0.6099999999999999, 'batch_size': 22.409821701049804}

Decentralized intervals search

7

10

7

12

3

12

7

12

12

12


Federated average best intervals

(0.15941918, {'clients': 10, 'skew': 0.1, 'act_fn': <function reluApprox at 0x7f7c88191280>, 'client_lr': 0.012, 'server_lr': 0.03162277660168379, 'client_momentum': 0.61, 'server_momentum': 0.9, 'batch_size': 22, 'epochs_per_round': 2, 'rounds': 30, 'runs': 1, 'intervals': [3, 5, 7, 10, 12], 'interval': 3})

(0.15938076, {'clients': 10, 'skew': 0.1, 'act_fn': <function reluApprox at 0x7f7c88191280>, 'client_lr': 0.012, 'server_lr': 0.03162277660168379, 'client_m